In [1]:
# %conda install conda-forge::opencadd

import os
import subprocess

# 导入必要的库
import warnings
from pathlib import Path

import nglview as nv
from openbabel import pybel
from opencadd.structure.core import Structure

In [2]:
# 定义路径
# 这里定义了当前工作目录和数据目录。数据目录用于存储中间和最终的结果文件。
HERE = Path(_dh[-1])
DATA = HERE / "data"
DATA.mkdir(exist_ok=True)
DATA

PosixPath('/work/run/projects/fanxi/learn/biorange/results/data')

In [3]:
# 下载 PDB 文件
def download_pdb(pdb_code: str, work_dir: str):
    import os

    import requests

    pdb_path = os.path.join(work_dir, f"{pdb_code}.pdb")

    if not os.path.exists(pdb_path):
        # 从蛋白质数据库 (PDB) 下载文件，如果文件不存在的话
        pdb_url = f"https://files.rcsb.org/download/{pdb_code}.pdb"
        response = requests.get(pdb_url)
        if response.status_code == 200:
            with open(pdb_path, "wt", encoding="utf-8") as pdb_file:
                pdb_file.write(response.text)

    return pdb_path if os.path.exists(pdb_path) else None

In [4]:


# 从字符串加载 PDB 文件
def from_string(cls, pdbid_or_path, **kwargs):
    if os.path.isfile(pdbid_or_path):
        return cls(pdbid_or_path, **kwargs)
    return cls.from_pdbid(pdbid_or_path, **kwargs)


# 将新的方法添加到 Structure 类中
Structure.from_string = classmethod(from_string)

In [5]:


# 从蛋白质数据库中检索结构
# 这里下载了指定的 PDB 文件并使用 Structure 类加载
pdb_id = "2ito"
download_pdb_dir = download_pdb(pdb_id, DATA)
structure = Structure.from_string(download_pdb_dir)
structure, download_pdb_dir

(<Universe with 2484 atoms>,
 '/work/run/projects/fanxi/learn/biorange/results/data/2ito.pdb')

In [6]:


# 将 PDB 文件转换为 PDBQT 文件
def pdb_to_pdbqt(pdb_path, pdbqt_path, pH=7.4):
    """
    Convert a PDB file to a PDBQT file needed by docking programs of the AutoDock family.

    Parameters
    ----------
    pdb_path: str or pathlib.Path
        Path to input PDB file.
    pdbqt_path: str or pathlib.path
        Path to output PDBQT file.
    pH: float
        Protonation at given pH.
    """
    molecule = list(pybel.readfile("pdb", str(pdb_path)))[0]
    # 在给定的 pH 值下添加氢
    molecule.OBMol.CorrectForPH(pH)
    molecule.addh()
    # 为每个原子添加部分电荷
    for atom in molecule.atoms:
        atom.OBAtom.GetPartialCharge()
    # 将结果写入 PDBQT 文件
    molecule.write("pdbqt", str(pdbqt_path), overwrite=True)
    return

In [7]:


# 将蛋白质转换为 PDBQT 格式
# 这里将蛋白质 PDB 文件转换为对接所需的 PDBQT 格式
pdb_to_pdbqt(download_pdb_dir, DATA / "protein.pdbqt")

*** Open Babel Warning  in PerceiveBondOrders
  Failed to kekulize aromatic bonds in OBMol::PerceiveBondOrders (title is /work/run/projects/fanxi/learn/biorange/results/data/2ito.pdb)



In [8]:


# 为蛋白质-配体复合物定义配体的 SMILES
# SMILES 是一种用于表示分子结构的字符串表示法
smiles = "COC1=C(C=C2C(=C1)N=CN=C2NC3=CC(=C(C=C3)F)Cl)OCCCN4CCOCC4"

In [9]:


# 将 SMILES 字符串转换为 PDBQT 文件
def smiles_to_pdbqt(smiles, pdbqt_path, pH=7.4):
    """
    Convert a SMILES string to a PDBQT file needed by docking programs of the AutoDock family.

    Parameters
    ----------
    smiles: str
        SMILES string.
    pdbqt_path: str or pathlib.path
        Path to output PDBQT file.
    pH: float
        Protonation at given pH.
    """
    molecule = pybel.readstring("smi", smiles)
    # 在给定的 pH 值下添加氢
    molecule.OBMol.CorrectForPH(pH)
    molecule.addh()
    # 生成 3D 坐标
    molecule.make3D(forcefield="mmff94s", steps=10000)
    # 为每个原子添加部分电荷
    for atom in molecule.atoms:
        atom.OBAtom.GetPartialCharge()
    # 将结果写入 PDBQT 文件
    molecule.write("pdbqt", str(pdbqt_path), overwrite=True)
    return

In [10]:


# 将配体转换为 PDBQT 格式
# 将配体的 SMILES 表示转换为 PDBQT 文件
smiles_to_pdbqt(smiles, DATA / "ligand.pdbqt")

In [11]:


# 定义结合口袋的中心和大小
# 这里根据配体的位置计算结合口袋的中心和大小
ligand_resname = "IRE"
ligand = structure.select_atoms(f"resname {ligand_resname}")
pocket_center = (ligand.positions.max(axis=0) + ligand.positions.min(axis=0)) / 2
pocket_size = ligand.positions.max(axis=0) - ligand.positions.min(axis=0) + 5

In [12]:


# 进行对接计算
def run_smina(
    ligand_path,
    protein_path,
    out_path,
    pocket_center,
    pocket_size,
    num_poses=10,
    exhaustiveness=8,
):
    """
    Perform docking with Smina.

    Parameters
    ----------
    ligand_path: str or pathlib.Path
        Path to ligand PDBQT file that should be docked.
    protein_path: str or pathlib.Path
        Path to protein PDBQT file that should be docked to.
    out_path: str or pathlib.Path
        Path to which docking poses should be saved, SDF or PDB format.
    pocket_center: iterable of float or int
        Coordinates defining the center of the binding site.
    pocket_size: iterable of float or int
        Lengths of edges defining the binding site.
    num_poses: int
        Maximum number of poses to generate.
    exhaustiveness: int
        Accuracy of docking calculations.

    Returns
    -------
    output_text: str
        The output of the Smina calculation.
    """
    # 使用 Smina 进行对接计算
    output_text = subprocess.check_output(
        [
            "smina",
            "--ligand",
            str(ligand_path),
            "--receptor",
            str(protein_path),
            "--out",
            str(out_path),
            "--center_x",
            str(pocket_center[0]),
            "--center_y",
            str(pocket_center[1]),
            "--center_z",
            str(pocket_center[2]),
            "--size_x",
            str(pocket_size[0]),
            "--size_y",
            str(pocket_size[1]),
            "--size_z",
            str(pocket_size[2]),
            "--num_modes",
            str(num_poses),
            "--exhaustiveness",
            str(exhaustiveness),
        ],
        universal_newlines=True,  # 需要捕获输出文本
    )
    return output_text

In [13]:


# 打印对接结果
# 进行对接计算并打印结果
output_text = run_smina(
    DATA / "ligand.pdbqt",
    DATA / "protein.pdbqt",
    DATA / "docking_poses.sdf",
    pocket_center,
    pocket_size,
)
print(output_text)

   _______  _______ _________ _        _______ 
  (  ____ \(       )\__   __/( (    /|(  ___  )
  | (    \/| () () |   ) (   |  \  ( || (   ) |
  | (_____ | || || |   | |   |   \ | || (___) |
  (_____  )| |(_)| |   | |   | (\ \) ||  ___  |
        ) || |   | |   | |   | | \   || (   ) |
  /\____) || )   ( |___) (___| )  \  || )   ( |
  \_______)|/     \|\_______/|/    )_)|/     \|


smina is based off AutoDock Vina. Please cite appropriately.

Weights      Terms
-0.035579    gauss(o=0,_w=0.5,_c=8)
-0.005156    gauss(o=3,_w=2,_c=8)
0.840245     repulsion(o=0,_c=8)
-0.035069    hydrophobic(g=0.5,_b=1.5,_c=8)
-0.587439    non_dir_h_bond(g=-0.7,_b=0,_c=8)
1.923        num_tors_div

Using random seed: -1008475330

0%   10   20   30   40   50   60   70   80   90   100%
|----|----|----|----|----|----|----|----|----|----|
***************************************************

mode |   affinity | dist from best mode
     | (kcal/mol) | rmsd l.b.| rmsd u.b.
-----+------------+----------+----------

In [14]:


# 检查输出文件是否已生成
# 确保对接结果文件已生成
(DATA / "docking_poses.sdf").exists()
# NBVAL_CHECK_OUTPUT

True

In [15]:
## 单独的亲和力提取
def extract_affinity(sdf_path):
    # 读取 SDF 文件
    sdf_molecules = pybel.readfile("sdf", sdf_path)

    # 提取亲和力信息
    affinities = []
    for mol in sdf_molecules:
        try:
            affinity = mol.data["minimizedAffinity"]  # Smina 的亲和力字段
            affinities.append(float(affinity))
        except KeyError:
            print("No affinity data found for one of the poses.")

    return affinities


# 示例使用
sdf_path = "data/docking_poses_2.sdf"
affinities = extract_affinity(sdf_path)
print(affinities)

[-6.36943722]


In [16]:


# 将 SDF 文件拆分为单独的分子文件
def split_sdf_file(sdf_path):
    """
    Split an SDF file into seperate files for each molecule.
    Each file is named with consecutive numbers.

    Parameters
    ----------
    sdf_path: str or pathlib.Path
        Path to SDF file that should be split.
    """
    sdf_path = Path(sdf_path)
    stem = sdf_path.stem
    parent = sdf_path.parent
    molecules = pybel.readfile("sdf", str(sdf_path))
    # 将每个分子写入单独的 SDF 文件
    for i, molecule in enumerate(molecules, 1):
        molecule.write("sdf", str(parent / f"{stem}_{i}.sdf"), overwrite=True)
    return

In [17]:


# 拆分 SDF 文件
# 将对接结果的 SDF 文件拆分为单独的分子文件
split_sdf_file(DATA / "docking_poses.sdf")

In [18]:
# 可视化对接结果
# 使用 NGLView 可视化对接结果
docking_pose_id = 2
view = nv.show_structure_file(
    str(DATA / f"docking_poses_{docking_pose_id}.sdf"),
    representations=[{"params": {}, "type": "licorice"}],
)
view.add_pdbid(pdb_id)
view

NGLWidget()

In [19]:
# 渲染图像
# 渲染对接结果的图像
view.render_image(trim=True, factor=2)

Image(value=b'', width='99%')

In [20]:
# 显示图像
# 显示渲染的图像
view._display_image()

In [21]:
view.download_image(filename="scre.png")

## 单独评分计算

In [22]:
!smina -r complex_0/6w70.pdb -l complex_0/rank1_confidence-1.18.sdf --score_only 

   _______  _______ _________ _        _______ 
  (  ____ \(       )\__   __/( (    /|(  ___  )
  | (    \/| () () |   ) (   |  \  ( || (   ) |
  | (_____ | || || |   | |   |   \ | || (___) |
  (_____  )| |(_)| |   | |   | (\ \) ||  ___  |
        ) || |   | |   | |   | | \   || (   ) |
  /\____) || )   ( |___) (___| )  \  || )   ( |
  \_______)|/     \|\_______/|/    )_)|/     \|


smina is based off AutoDock Vina. Please cite appropriately.

Weights      Terms
-0.035579    gauss(o=0,_w=0.5,_c=8)
-0.005156    gauss(o=3,_w=2,_c=8)
0.840245     repulsion(o=0,_c=8)
-0.035069    hydrophobic(g=0.5,_b=1.5,_c=8)
-0.587439    non_dir_h_bond(g=-0.7,_b=0,_c=8)
1.923        num_tors_div

*** Open Babel Warning  in Init
  Unable to open data file 'space-groups.txt'
*** Open Babel Warning  in Init
  Cannot initialize database 'space-groups.txt' which may cause further errors.
## Name gauss(o=0,_w=0.5,_c=8) gauss(o=3,_w=2,_c=8) repulsion(o=0,_c=8) hydrophobic(g=0.5,_b=1.5,_c=8) non_dir_h_bond(g=-0.

## 单独的可视化

In [24]:
import py3Dmol

def visualize_molecule_complex(sdf_file, pdb_file, width=1200, height=900):
    viewer = py3Dmol.view(width=width, height=height)

    # 加载蛋白质和小分子
    with open(pdb_file, "r") as f:
        viewer.addModel(f.read(), "pdb")
    with open(sdf_file, "r") as f:
        viewer.addModel(f.read(), "sdf")

    # 设置蛋白质样式
    viewer.setStyle(
        {"model": 0},  
        {
            "cartoon": {"color": "spectrum", "opacity": 1},
            "surface": {"opacity": 1},
        },
    )

    # 设置小分子样式
    viewer.setStyle(
        {"model": 1},
        {
            "stick": {"colorscheme": "blueCarbon","radius": 0.2},
            "sphere": {"colorscheme": "blueCarbon", "radius": 0.5},
        },
    )
    # 外层棉花糖效果
    viewer.addSurface(
        {"model": 0},
        {
            "opacity": 0.6,
            "color": "white",
            "opacityType": "maximum",
            "colorType": "maximum",
            "style": "surface",
            "wireframe": False,
        },
    )
    # # 添加相互作用（这需要您知道具体的相互作用位置）
    # viewer.addLine({"start": {"x": 0, "y": 0, "z": 0}, "end": {"x": 3, "y": 4, "z": 5}})

    # # 添加标签（这需要您知道具体的残基位置）
    # viewer.addLabel(
    #     "Lys101", {"position": {"x": 10, "y": 20, "z": 30}, "backgroundColor": "white"}
    # )
    # # 添加高亮区域，电荷分布等等 都需要相应的数据或者计算
    viewer.zoomTo()
    return viewer


# 使用示例
viewer = visualize_molecule_complex(
    "complex_0/rank1_confidence-1.18.sdf", "complex_0/6w70.pdb"
)
viewer.show()

3Dmol.js failed to load for some reason. Please check your browser console for error messages.